In [ ]:
!pip install --no-index /kaggle/input/imc2024-packages-lightglue-rerun-kornia/* --no-deps
!mkdir -p /root/.cache/torch/hub/checkpoints
!cp /kaggle/input/aliked/pytorch/aliked-n16/1/aliked-n16.pth /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/lightglue/pytorch/aliked/1/aliked_lightglue.pth /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/lightglue/pytorch/aliked/1/aliked_lightglue.pth /root/.cache/torch/hub/checkpoints/aliked_lightglue_v0-1_arxiv-pth

In [ ]:
!pip install umap-learn
!pip install hdbscan

In [1]:
import torch
import torch.nn.functional as F
import kornia as K
import kornia.feature as KF
import h5py
import dataclasses
from IPython.display import clear_output
from collections import defaultdict
from copy import deepcopy
from lightglue import match_pair
from lightglue import ALIKED, LightGlue
from lightglue.utils import load_image, rbd
from transformers import AutoImageProcessor, AutoModel
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import os
import cv2 as cv
from tqdm import tqdm
from time import time, sleep
import umap
from hdbscan import HDBSCAN
import gc
import pandas as pd

import pycolmap
import sys
sys.path.append('/kaggle/input/imc25-utils')
from database import *
from h5_to_db import *
import metric

/usr/local/lib/python3.10/dist-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/usr/local/lib/python3.10/dist-packages/lightglue/lightglue.py:24: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


In [2]:
device = K.utils.get_cuda_device_if_available(0)
print(f'{device=}')

device=device(type='cuda', index=0)


In [3]:
data_dir = "/kaggle/input/image-matching-challenge-2025"
train_labels = pd.read_csv(f"{data_dir}/train_labels.csv")
train_labels

,dataset,scene,image,rotation_matrix,translation_vector
0,imc2023_haiper,fountain,fountain_image_116.png,0.122655949;0.947713775;-0.294608417;0.1226706...,0.093771314;-0.803560988;2.062001533
1,imc2023_haiper,fountain,fountain_image_108.png,0.474305910;0.359108654;-0.803787832;0.2888416...,0.358946647;-0.797557548;1.910906929
2,imc2023_haiper,fountain,fountain_image_101.png,0.565115476;-0.138485064;-0.813305838;0.506678...,0.146922468;-0.981392596;2.009002852
3,imc2023_haiper,fountain,fountain_image_082.png,-0.308320392;-0.794654112;0.522937261;0.948141...,0.206413831;-1.174321103;3.667167680
4,imc2023_haiper,fountain,fountain_image_071.png,-0.569002830;-0.103808175;0.815757098;0.778745...,-0.015140892;-1.334052012;3.488936597
...,...,...,...,...,...
1940,stairs,stairs_split_2,stairs_split_2_1710453733751.png,0.961762441;-0.187990401;0.199179859;-0.177691...,-0.112850000;-3.521750000;-2.859750000
1941,stairs,stairs_split_2,stairs_split_2_1710453759963.png,0.237960308;0.580896704;-0.778417569;0.4077886...,-0.490768000;-3.064140000;3.008420000
1942,stairs,stairs_split_2,stairs_split_2_1710453805788.png,0.309067298;0.541767194;-0.781642957;0.4038963...,-0.572757000;0.885835000;4.987270000
1943,stairs,stairs_split_2,stairs_split_2_1710453765165.png,0.301920210;0.609614467;-0.732949103;0.5007116...,-0.135613000;-1.832910000;1.598790000


In [4]:
def pick_images(image_dir, datasets=[]):
    image_paths = []
    if len(datasets) == 0:
        df = train_labels
    else:
        df = train_labels[train_labels["dataset"].isin(datasets)]
    for _, row in df.iterrows():
        path = os.path.join(image_dir, row["dataset"],row["image"])
        image_paths.append(path)
    return image_paths

In [5]:
def get_clip_embeddings(image_paths, device):
    # Load Pretrained CLIP
    model_name = "openai/clip-vit-base-patch32"
    clip_model = CLIPModel.from_pretrained(model_name).to(device)
    processor = CLIPProcessor.from_pretrained(model_name)

    def get_image_embeddings(image_path: str):
        image = Image.open(image_path)
    
        with torch.no_grad():
            inputs = processor(images=image, return_tensors="pt", padding=True).to(device)
            return clip_model.get_image_features(**inputs)

    # Get image embeddings using CLIP
    image_embeddings = torch.cat([get_image_embeddings(image_path) for image_path in image_paths])

    # Dimensionality reduction
    umap_model = umap.UMAP(n_components = 5)
    reduced_embeddings = umap_model.fit_transform(image_embeddings.detach().cpu().numpy())

    # Cleanup
    del clip_model
    del image_embeddings
    torch.cuda.empty_cache()
    gc.collect()

    return reduced_embeddings

def cluster_images(reduced_embeddings):
    # Clustering
    clusterer = HDBSCAN(min_cluster_size=3)
    clusterer.fit(reduced_embeddings)
    clusters = clusterer.labels_
    return clusters

In [ ]:
def load_torch_image(fname, device=torch.device('cpu')):
    img = K.io.load_image(fname, K.io.ImageLoadType.RGB32, device=device)[None, ...]
    return img
    
def get_img_pairs_exhaustive(img_fnames):
    index_pairs = []
    for i in range(len(img_fnames)):
        for j in range(i+1, len(img_fnames)):
            index_pairs.append((i,j))
    return index_pairs


def get_image_pairs_shortlist(fnames,
                              sim_th = 0.6, # should be strict
                              min_pairs = 30,
                              exhaustive_if_less = 20,
                              device=torch.device('cpu')):
    num_imgs = len(fnames)
    if num_imgs <= exhaustive_if_less:
        return get_img_pairs_exhaustive(fnames)
    descs = torch.tensor(get_clip_embeddings(fnames, device))
    dm = torch.cdist(descs, descs, p=2).numpy()
    # removing half
    mask = dm <= sim_th
    total = 0
    matching_list = []
    ar = np.arange(num_imgs)
    already_there_set = []
    for st_idx in range(num_imgs-1):
        mask_idx = mask[st_idx]
        to_match = ar[mask_idx]
        if len(to_match) < min_pairs:
            to_match = np.argsort(dm[st_idx])[:min_pairs]  
        for idx in to_match:
            if st_idx == idx:
                continue
            if dm[st_idx, idx] < 1000:
                matching_list.append(tuple(sorted((st_idx, idx.item()))))
                total+=1
    matching_list = sorted(list(set(matching_list)))
    return matching_list

def detect_aliked(img_fnames,
                  feature_dir = '.featureout',
                  num_features = 4096,
                  resize_to = 1024,
                  device=torch.device('cpu')):
    dtype = torch.float32 # ALIKED has issues with float16
    extractor = ALIKED(max_num_keypoints=num_features, detection_threshold=0.01, resize=resize_to).eval().to(device, dtype)
    if not os.path.isdir(feature_dir):
        os.makedirs(feature_dir)
    with h5py.File(f'{feature_dir}/keypoints.h5', mode='w') as f_kp, \
         h5py.File(f'{feature_dir}/descriptors.h5', mode='w') as f_desc:
        for img_path in tqdm(img_fnames):
            img_fname = img_path.split('/')[-1]
            key = img_fname
            with torch.inference_mode():
                image0 = load_torch_image(img_path, device=device).to(dtype)
                feats0 = extractor.extract(image0)  # auto-resize the image, disable with resize=None
                kpts = feats0['keypoints'].reshape(-1, 2).detach().cpu().numpy()
                descs = feats0['descriptors'].reshape(len(kpts), -1).detach().cpu().numpy()
                f_kp[key] = kpts
                f_desc[key] = descs
    return

def match_with_lightglue(img_fnames,
                   index_pairs,
                   feature_dir = '.featureout',
                   device=torch.device('cpu'),
                   min_matches=25,verbose=True):
    lg_matcher = KF.LightGlueMatcher("aliked", {"width_confidence": -1,
                                                "depth_confidence": -1,
                                                 "mp": True if 'cuda' in str(device) else False}).eval().to(device)
    with h5py.File(f'{feature_dir}/keypoints.h5', mode='r') as f_kp, \
        h5py.File(f'{feature_dir}/descriptors.h5', mode='r') as f_desc, \
        h5py.File(f'{feature_dir}/matches.h5', mode='w') as f_match:
        for pair_idx in tqdm(index_pairs):
            idx1, idx2 = pair_idx
            fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
            key1, key2 = fname1.split('/')[-1], fname2.split('/')[-1]
            kp1 = torch.from_numpy(f_kp[key1][...]).to(device)
            kp2 = torch.from_numpy(f_kp[key2][...]).to(device)
            desc1 = torch.from_numpy(f_desc[key1][...]).to(device)
            desc2 = torch.from_numpy(f_desc[key2][...]).to(device)
            with torch.inference_mode():
                dists, idxs = lg_matcher(desc1,
                                         desc2,
                                         KF.laf_from_center_scale_ori(kp1[None]),
                                         KF.laf_from_center_scale_ori(kp2[None]))
            if len(idxs)  == 0:
                continue
            n_matches = len(idxs)
            if verbose:
                print (f'{key1}-{key2}: {n_matches} matches')
            group  = f_match.require_group(key1)
            if n_matches >= min_matches:
                 group.create_dataset(key2, data=idxs.detach().cpu().numpy().reshape(-1, 2))
    return

def import_into_colmap(img_dir, feature_dir ='.featureout', database_path = 'colmap.db'):
    db = COLMAPDatabase.connect(database_path)
    db.create_tables()
    single_camera = False
    fname_to_id = add_keypoints(db, feature_dir, img_dir, '', 'simple-pinhole', single_camera)
    add_matches(
        db,
        feature_dir,
        fname_to_id,
    )
    db.commit()
    return

In [6]:
@dataclasses.dataclass
class Prediction:
    image_id: str | None  # A unique identifier for the row -- unused otherwise. Used only on the hidden test set.
    dataset: str
    filename: str
    cluster_index: int | None = None
    rotation: np.ndarray | None = None
    translation: np.ndarray | None = None

In [7]:
is_train = True # Set to False if submitting to contest
workdir = '/kaggle/working/result/'
os.makedirs(workdir, exist_ok=True)

if is_train:
    sample_submission_csv = os.path.join(data_dir, 'train_labels.csv')
else:
    sample_submission_csv = os.path.join(data_dir, 'sample_submission.csv')

samples = {}
competition_data = pd.read_csv(sample_submission_csv)
for _, row in competition_data.iterrows():
    # Note: For the test data, the "scene" column has no meaning, and the rotation_matrix and translation_vector columns are random.
    if row.dataset not in samples:
        samples[row.dataset] = []
    samples[row.dataset].append(
        Prediction(
            image_id=None if is_train else row.image_id,
            dataset=row.dataset,
            filename=row.image
        )
    )

for dataset in samples:
    print(f'Dataset "{dataset}" -> num_images={len(samples[dataset])}')

Dataset "imc2023_haiper" -> num_images=54
Dataset "imc2023_heritage" -> num_images=209
Dataset "imc2023_theather_imc2024_church" -> num_images=76
Dataset "imc2024_dioscuri_baalshamin" -> num_images=138
Dataset "imc2024_lizard_pond" -> num_images=214
Dataset "pt_brandenburg_british_buckingham" -> num_images=225
Dataset "pt_piazzasanmarco_grandplace" -> num_images=168
Dataset "pt_sacrecoeur_trevi_tajmahal" -> num_images=225
Dataset "pt_stpeters_stpauls" -> num_images=200
Dataset "amy_gardens" -> num_images=200
Dataset "fbk_vineyard" -> num_images=163
Dataset "ETs" -> num_images=22
Dataset "stairs" -> num_images=51


## Visualize clusters

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import random

def visualize_clusters(umap_embeddings, cluster_labels, image_paths, num_samples=5):
    """
    Visualizes UMAP-reduced embeddings with HDBSCAN clusters and displays example images from a random cluster.
    
    Args:
        umap_embeddings (numpy.ndarray): 2D UMAP-reduced embeddings (N x 2).
        cluster_labels (numpy.ndarray): Cluster assignments for each image (N,).
        image_paths (list): List of file paths to the images (N,).
        num_samples (int): Number of images to display from a selected cluster.
    """

    unique_clusters = set(cluster_labels)
    palette = sns.color_palette("husl", len(unique_clusters))  # Generate colors for clusters
    
    # Scatter plot of clusters
    plt.figure(figsize=(10, 6))
    for cluster_id in unique_clusters:
        indices = np.where(cluster_labels == cluster_id)[0]
        plt.scatter(umap_embeddings[indices, 0], umap_embeddings[indices, 1], 
                    label=f"Cluster {cluster_id}" if cluster_id != -1 else "Outliers",
                    alpha=0.6, edgecolors='k', s=30, color=palette[cluster_id % len(palette)])
    
    plt.xlabel("UMAP Dimension 1")
    plt.ylabel("UMAP Dimension 2")
    plt.title("UMAP Visualization of Image Clusters")
    plt.legend()
    plt.show()

    # Pick a random cluster (excluding outliers, labeled as -1)
    valid_clusters = [c for c in unique_clusters if c != -1]
    if not valid_clusters:
        print("No valid clusters found.")
        return

    selected_cluster = random.choice(valid_clusters)
    print(f"\nDisplaying {num_samples} images from Cluster {selected_cluster}:")

    # Get image paths from the selected cluster
    cluster_indices = np.where(cluster_labels == selected_cluster)[0]
    sample_indices = random.sample(list(cluster_indices), min(num_samples, len(cluster_indices)))
    sample_images = [image_paths[i] for i in sample_indices]

    # Display images
    fig, axes = plt.subplots(1, len(sample_images), figsize=(15, 5))
    for ax, img_path in zip(axes, sample_images):
        img = Image.open(img_path)
        ax.imshow(img)
        ax.axis("off")
    plt.show()

In [ ]:
# Clustering example on imc2023_haiper
datasets = ["imc2023_haiper"]
image_paths = pick_images(image_dir, datasets)
reduced_embeddings = get_clip_embeddings(image_paths, device)
clusters = cluster_images(reduced_embeddings)
visualize_clusters(reduced_embeddings, clusters, image_paths)

## Run COLMAP for Camera Pose Estimation

In [ ]:
datasets_to_process = None #Run on all test datasets
if is_train:
    # Note: When running on the training dataset, the notebook will hit the time limit and die. Use this filter to run on a few specific datasets.
    datasets_to_process = [
    	# New data.
    	'amy_gardens',
    	'ETs',
    	'fbk_vineyard',
    	'stairs',
    	# Data from IMC 2023 and 2024.
    	# 'imc2024_dioscuri_baalshamin',
    	# 'imc2023_theather_imc2024_church',
    	# 'imc2023_heritage',
    	# 'imc2023_haiper',
    	# 'imc2024_lizard_pond',
    	# Crowdsourced PhotoTourism data.
    	# 'pt_stpeters_stpauls',
    	#'pt_brandenburg_british_buckingham',
    	# 'pt_piazzasanmarco_grandplace',
    	# 'pt_sacrecoeur_trevi_tajmahal',
    ]

timings = {
    "shortlisting":[],
    "feature_detection": [],
    "feature_matching":[],
    "RANSAC": [],
    "Reconstruction": [],
}
mapping_result_strs = []


print (f"Extracting on device {device}")
for dataset, predictions in samples.items():
    if datasets_to_process and dataset not in datasets_to_process:
        print(f'Skipping "{dataset}"')
        continue
    
    images_dir = os.path.join(data_dir, 'train' if is_train else 'test', dataset)
    images = [os.path.join(images_dir, p.filename) for p in predictions]

    print(f'\nProcessing dataset "{dataset}": {len(images)} images')

    filename_to_index = {p.filename: idx for idx, p in enumerate(predictions)}

    feature_dir = os.path.join(workdir, 'featureout', dataset)
    os.makedirs(feature_dir, exist_ok=True)

    # Wrap algos in try-except blocks so we can populate a submission even if one scene crashes.
    try:
        t = time()
        index_pairs = get_image_pairs_shortlist(
            images,
            sim_th = 0.3, # should be strict
            min_pairs = 20, # we should select at least min_pairs PER IMAGE with biggest similarity
            exhaustive_if_less = 20,
            device=device
        )
        timings['shortlisting'].append(time() - t)
        print (f'Shortlisting. Number of pairs to match: {len(index_pairs)}. Done in {time() - t:.4f} sec')
        gc.collect()
    
        t = time()

        detect_aliked(images, feature_dir, 4096, device=device)
        gc.collect()
        timings['feature_detection'].append(time() - t)
        print(f'Features detected in {time() - t:.4f} sec')
        
        t = time()
        match_with_lightglue(images, index_pairs, feature_dir=feature_dir, device=device, verbose=False)
        timings['feature_matching'].append(time() - t)
        print(f'Features matched in {time() - t:.4f} sec')

        database_path = os.path.join(feature_dir, 'colmap.db')
        if os.path.isfile(database_path):
            os.remove(database_path)
        gc.collect()
        sleep(1)
        import_into_colmap(images_dir, feature_dir=feature_dir, database_path=database_path)
        output_path = f'{feature_dir}/colmap_rec_aliked'
        
        t = time()
        pycolmap.match_exhaustive(database_path)
        timings['RANSAC'].append(time() - t)
        print(f'Ran RANSAC in {time() - t:.4f} sec')
        
        # By default colmap does not generate a reconstruction if less than 10 images are registered.
        # Lower it to 3.
        mapper_options = pycolmap.IncrementalPipelineOptions()
        mapper_options.min_model_size = 3
        mapper_options.max_num_models = 25
        os.makedirs(output_path, exist_ok=True)
        t = time()
        maps = pycolmap.incremental_mapping(
            database_path=database_path, 
            image_path=images_dir,
            output_path=output_path,
            options=mapper_options)
        sleep(1)
        timings['Reconstruction'].append(time() - t)
        print(f'Reconstruction done in  {time() - t:.4f} sec')
        print(maps)

        clear_output(wait=False)
    
        registered = 0
        for map_index, cur_map in maps.items():
            for index, image in cur_map.images.items():
                prediction_index = filename_to_index[image.name]
                predictions[prediction_index].cluster_index = map_index
                predictions[prediction_index].rotation = deepcopy(image.cam_from_world.rotation.matrix())
                predictions[prediction_index].translation = deepcopy(image.cam_from_world.translation)
                registered += 1
        mapping_result_str = f'Dataset "{dataset}" -> Registered {registered} / {len(images)} images with {len(maps)} clusters'
        mapping_result_strs.append(mapping_result_str)
        print(mapping_result_str)
        gc.collect()
    except Exception as e:
        print(e)
        # raise e
        mapping_result_str = f'Dataset "{dataset}" -> Failed!'
        mapping_result_strs.append(mapping_result_str)
        print(mapping_result_str)

print('\nResults')
for s in mapping_result_strs:
    print(s)

print('\nTimings')
for k, v in timings.items():
    print(f'{k} -> total={sum(v):.02f} sec.')

## Run Fast3R for camera pose estimation

In [ ]:
# Download PyTorch3D from wheel
import sys
import torch
pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])
print(version_str)
!pip install -q iopath
!pip install -q --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html

In [ ]:
# clone Fast3R repo
!git clone https://github.com/facebookresearch/fast3r
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu125

# install requirements
!pip install -q -r /kaggle/working/fast3r/requirements.txt

# install fast3r as a package (so you can import fast3r and use it in your own project)
!pip install -q -e /kaggle/working/fast3r

In [ ]:
# Debugging in case you get ModuleNotFoundError
import os
import pkgutil
import fast3r
import sys
print(sys.path)
print([p for p in sys.path if 'fast3r' in p])
print([k for k in sys.modules if k.startswith('fast3r')])

for sub in pkgutil.iter_modules(fast3r.__path__):
    print(sub)


In [33]:
import torch
from fast3r.models.fast3r import Fast3R
from fast3r.models.multiview_dust3r_module import MultiViewDUSt3RLitModule
from fast3r.dust3r.utils.image import load_images
from fast3r.dust3r.inference_multiview import inference
import numpy as np
import gc

def camera_pose_estimation(clusters, dataset, device):
    predictions = samples[dataset]
    filename_to_index = {p.filename: idx for idx, p in enumerate(predictions)}
    # Load the model from Hugging Face
    model = Fast3R.from_pretrained("jedyang97/Fast3R_ViT_Large_512")
    model.encoder_args["attn_implementation"] = "pytorch_naive"
    model.build_encoder(model.encoder_args)
    model.decoder_args["attn_implementation"] = "pytorch_naive"
    model.build_decoder(model.decoder_args)
    
    model = model.to(device)
    lit_module = MultiViewDUSt3RLitModule.load_for_inference(model)
    
    model.eval()
    lit_module.eval()
    for cluster in clusters:
        if cluster == "outliers":
            # Skip inference for outliers
            for outlier in clusters[cluster]:
                prediction_index = filename_to_index[os.path.basename(outlier)]
                predictions[prediction_index].cluster_index = None
            continue
            
        # Load images
        images = load_images(clusters[cluster], size=384, verbose=False)
    
        # Run inference
        try:
            output_dict = inference(
                images,
                model,
                device,
                dtype=torch.float32,
                verbose=False,
                profiling=False
            )
        except Exception as e:
            print(e)
            torch.cuda.empty_cache()
            gc.collect()
            continue
        
        # Extract camera poses
        poses_c2w_batch, estimated_focals = MultiViewDUSt3RLitModule.estimate_camera_poses(
            output_dict['preds'],
            niter_PnP=100,
            focal_length_estimation_method='first_view_from_global_head'
        )
        # poses_c2w_batch is a list; the first element contains the estimated poses for each view.
        camera_poses = poses_c2w_batch[0]

        # Record rotation and translation
        for pose, image_path in zip(camera_poses, clusters[cluster]):
            prediction_index = filename_to_index[os.path.basename(image_path)]
    
            predictions[prediction_index].cluster_index = cluster
            predictions[prediction_index].rotation = deepcopy(pose[:3, :3])
            predictions[prediction_index].translation = deepcopy(pose[:3, 3])
    
        # Release GPU memory when done
        del poses_c2w_batch
        torch.cuda.empty_cache()
        gc.collect()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
# Evaluation on clusters found using CLIP ViT encoder

#datasets = list(train_labels["dataset"].unique()) # All datasets in train dir
datasets = ["ETs"]
data = []
images_dir = os.path.join(data_dir, 'train' if is_train else 'test')
for dataset in datasets:
    image_paths = pick_images(images_dir, datasets=[dataset])
    print(f"Dataset: {dataset} with {len(image_paths)} images.")
    print("Clustering images...")
    reduced_embeddings = get_clip_embeddings(image_paths, device)
    clusters = cluster_images(reduced_embeddings)
    clusters_dict = {} 
    for path, cluster_name in zip(image_paths, clusters):
        if cluster_name == -1:
            cluster_name = "outliers"
        if cluster_name not in clusters_dict:
            clusters_dict[cluster_name] = []
        clusters_dict[cluster_name].append(path)
    print(f"Found {len(clusters_dict)} clusters.")

    print("Estimating camera poses for each cluster...")
    camera_pose_estimation(clusters_dict, dataset, device)

Dataset: ETs with 22 images.
Clustering images...
Found 2 clusters.
Estimating camera poses for each cluster...


## Save data to CSV and Evaluate

In [35]:
# Must Create a submission file.

array_to_str = lambda array: ';'.join([f"{x:.09f}" for x in array])
none_to_str = lambda n: ';'.join(['nan'] * n)

submission_file = '/kaggle/working/submission.csv'
with open(submission_file, 'w') as f:
    if is_train:
        f.write('dataset,scene,image,rotation_matrix,translation_vector\n')
        for dataset in samples:
            for prediction in samples[dataset]:
                cluster_name = 'outliers' if prediction.cluster_index is None else f'cluster{prediction.cluster_index}'
                rotation = none_to_str(9) if prediction.rotation is None else array_to_str(prediction.rotation.flatten())
                translation = none_to_str(3) if prediction.translation is None else array_to_str(prediction.translation)
                f.write(f'{prediction.dataset},{cluster_name},{prediction.filename},{rotation},{translation}\n')
    else:
        f.write('image_id,dataset,scene,image,rotation_matrix,translation_vector\n')
        for dataset in samples:
            for prediction in samples[dataset]:
                cluster_name = 'outliers' if prediction.cluster_index is None else f'cluster{prediction.cluster_index}'
                rotation = none_to_str(9) if prediction.rotation is None else array_to_str(prediction.rotation.flatten())
                translation = none_to_str(3) if prediction.translation is None else array_to_str(prediction.translation)
                f.write(f'{prediction.image_id},{prediction.dataset},{cluster_name},{prediction.filename},{rotation},{translation}\n')

!head {submission_file}

dataset,scene,image,rotation_matrix,translation_vector
imc2023_haiper,outliers,fountain_image_116.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_108.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_101.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_082.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_071.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_025.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_000.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_007.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_012.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
if is_train:
    t = time()
    final_score, dataset_scores = metric.score(
        gt_csv='/kaggle/input/image-matching-challenge-2025/train_labels.csv',
        user_csv=submission_file,
        thresholds_csv='/kaggle/input/image-matching-challenge-2025/train_thresholds.csv',
        mask_csv=None if is_train else os.path.join(data_dir, 'mask.csv'),
        inl_cf=0,
        strict_cf=-1,
        verbose=True,
    )
    print(f'Computed metric in: {time() - t:.02f} sec.')

imc2023_haiper: score=0.00% (mAA=0.00%, clusterness=0.00%)
imc2023_heritage: score=0.00% (mAA=0.00%, clusterness=0.00%)
imc2023_theather_imc2024_church: score=0.00% (mAA=0.00%, clusterness=0.00%)
imc2024_dioscuri_baalshamin: score=0.00% (mAA=0.00%, clusterness=0.00%)
imc2024_lizard_pond: score=0.00% (mAA=0.00%, clusterness=0.00%)
pt_brandenburg_british_buckingham: score=0.00% (mAA=0.00%, clusterness=0.00%)
pt_piazzasanmarco_grandplace: score=0.00% (mAA=0.00%, clusterness=0.00%)
pt_sacrecoeur_trevi_tajmahal: score=0.00% (mAA=0.00%, clusterness=0.00%)
pt_stpeters_stpauls: score=0.00% (mAA=0.00%, clusterness=0.00%)


/kaggle/input/imc25-utils/metric.py:293: RuntimeWarning: divide by zero encountered in scalar divide
  M[:ndims, :ndims] *= math.sqrt(np.sum(v1) / np.sum(v0))
/kaggle/input/imc25-utils/metric.py:293: RuntimeWarning: invalid value encountered in multiply
  M[:ndims, :ndims] *= math.sqrt(np.sum(v1) / np.sum(v0))
/kaggle/input/imc25-utils/metric.py:175: RuntimeWarning: invalid value encountered in less
  inl = np.expand_dims(err, axis=1) < ransac_threshold2


amy_gardens: score=1.51% (mAA=0.76%, clusterness=100.00%)
fbk_vineyard: score=0.00% (mAA=0.00%, clusterness=0.00%)
ETs: score=0.00% (mAA=0.00%, clusterness=86.36%)
stairs: score=0.00% (mAA=0.00%, clusterness=0.00%)
Average over all datasets: score=0.12% (mAA=0.06%, clusterness=14.34%)
Computed metric in: 127.01 sec.
